Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

# Text2SQL with AutoGen & Azure OpenAI

This notebook demonstrates how the AutoGen Agents can be integrated with Azure OpenAI to answer questions from the database based on the schemas provided. 

A multi-shot approach is used for SQL generation for more reliable results and reduced token usage. More details can be found in the README.md.

In [ ]:
import dotenv
import logging
from autogen_agentchat.task import Console
from agentic_text_2_sql import AgenticText2Sql

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
dotenv.load_dotenv()

## Bot setup

In [ ]:
agentic_text_2_sql = AgenticText2Sql(target_engine="TSQL", engine_specific_rules="Use TOP X to limit the number of rows returned instead of LIMIT X. NEVER USE LIMIT X as it produces a syntax error.").agentic_flow

In [ ]:
result = agentic_text_2_sql.run_stream(task="What are the total number of sales within 2008?")

In [ ]:
await Console(result)